In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer

C:\Users\91932\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
model = AutoModelForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)


C:\Users\91932\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\91932\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Some weights of the model checkpoint at bert-base-uncas

In [3]:
import pandas as pd
# from rouge import Rouge
# from transformers import pipeline

# Load the sentiment-response dataset
df = pd.read_excel('result.xlsx')
sentiments = df['sentiment'].tolist()
emails = df['response'].tolist()

In [4]:
sentiments

['positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative']

In [6]:
for i in range(len(sentiments)):
    if sentiments[i]=='positive':
        sentiments[i]=1
    else:
        sentiments[i]=0
        

In [7]:
sentiments

[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

In [22]:
inputs = tokenizer(emails, padding=True, truncation=True, return_tensors='pt')
labels = torch.tensor(sentiments)

In [27]:
inputs[0]

Encoding(num_tokens=151, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])

In [26]:
len(inputs[2])

151

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer

# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
model = AutoModelForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

# Load and preprocess the dataset
# Replace this with code to load your own dataset
# emails = ['This is a positive email', 'This is a negative email', ...]
# sentiments = [1, 0, ...]  # 1 = positive, 0 = negative
inputs = tokenizer(emails[:69], padding=True, truncation=True, return_tensors='pt')
labels = torch.tensor(sentiments[:69])

# Fine-tune the model
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    evaluation_strategy='epoch',     # evaluation strategy
    num_train_epochs=3,              # total number of training epochs
    learning_rate=2e-5,              # learning rate
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=inputs,
    eval_dataset=inputs,
    compute_metrics=lambda pred: {'accuracy': (pred.predictions.argmax(axis=1) == labels).float().mean().item()}
)
trainer.train()

# Save the model
model.save_pretrained('my_model')

# Generate text based on sentiment
def generate_text(sentiment):
    # Load the model
    model = AutoModelForSequenceClassification.from_pretrained('my_model')
    model.eval()

    # Generate text
    prompt = 'This email is '
    if sentiment == 'positive':
        prompt += 'positive.'
    else:
        prompt += 'negative.'
    input_ids = tokenizer.encode(prompt, return_tensors='pt')
    output = model.generate(input_ids=input_ids, max_length=100)

    # Decode the output
    return tokenizer.decode(output[0], skip_special_tokens=True)

# Evaluate the generated text
generated_text = generate_text('positive')
print(generated_text)


In [16]:


# Fine-tune the model
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    evaluation_strategy='epoch',     # evaluation strategy
    num_train_epochs=10,              # total number of training epochs
    learning_rate=2e-5,              # learning rate
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=inputs,
    eval_dataset=inputs,
    compute_metrics=lambda pred: {'accuracy': (pred.predictions.argmax(axis=1) == labels).float().mean().item()}
)
trainer.train()

# Save the model
model.save_pretrained('my_model')

C:\Users\91932\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\transformers\optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(





ValueError: The batch received was empty, your model won't be able to train on it. Double-check that your training dataset contains keys expected by the model: input_ids,attention_mask,token_type_ids,position_ids,head_mask,inputs_embeds,labels,output_attentions,output_hidden_states,return_dict,labels,label_ids,label.

In [ ]:


# Generate text based on sentiment
def generate_text(sentiment):
    # Load the model
    model = AutoModelForSequenceClassification.from_pretrained('my_model')
    model.eval()

    # Generate text
    prompt = 'This email is '
    if sentiment == 'positive':
        prompt += 'positive.'
    else:
        prompt += 'negative.'
    input_ids = tokenizer.encode(prompt, return_tensors='pt')
    output = model.generate(input_ids=input_ids, max_length=100)

    # Decode the output
    return tokenizer.decode(output[0], skip_special_tokens=True)

# Evaluate the generated text
generated_text = generate_text('positive')
print(generated_text)


In [17]:
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
import torch

# Load the tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

# Prepare the dataset
inputs = []
labels = []

for i in range(1, 70):
    with open(f'email_{i}.txt', 'r') as f:
        email_text = f.read().replace('\n', '')
    with open(f'label_{i}.txt', 'r') as f:
        label = f.read().replace('\n', '')
    input_ids = tokenizer.encode(email_text, add_special_tokens=True)
    attention_mask = [1] * len(input_ids)
    inputs.append({
        'input_ids': input_ids,
        'attention_mask': attention_mask
    })
    labels.append(int(label))

# Convert the dataset to tensors
inputs = [torch.tensor(x['input_ids']) for x in inputs]
masks = [torch.tensor(x['attention_mask']) for x in inputs]
inputs = [(input, mask) for input, mask in zip(inputs, masks)]
labels = torch.tensor(labels)

# Fine-tune the model
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    evaluation_strategy='epoch',     # evaluation strategy
    num_train_epochs=3,              # total number of training epochs
    learning_rate=2e-5,              # learning rate
    per_device_train_batch_size=4,   # batch size per device during training
    per_device_eval_batch_size=16,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=list(zip(inputs, labels)),
    eval_dataset=list(zip(inputs, labels)),
    compute_metrics=lambda pred: {'accuracy': (pred.predictions.argmax(axis=1) == labels).float().mean().item()}
)
trainer.train()

# Save the model
model.save_pretrained('my_model')


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

FileNotFoundError: [Errno 2] No such file or directory: 'email_1.txt'